In [1]:
import pandas as pd
import numpy as np
import qiime2 as q2

import biom
import h5py

from qiime2.plugins import feature_table, diversity, greengenes2, taxa

## Metadata - General from LIBR

In [2]:
# diagnosis metadata from LIBR 
libr_md = pd.read_csv('../Updated_Data/DukeCollab_Metadata-9-22-2022.csv', index_col=0)

# medications metadata - ATC format 
medications = pd.read_csv('../Updated_Data/Tulsa_medications_ATC_v1.csv', 
                         encoding='unicode_escape', index_col=0)
medications = medications.loc[medications.atc.notna()]

# ATC medications of interest
atc_medication_key = ['C10_LipidReducingAgents', 'J01_Antibacterials',
                      'N05_Psycholeptics', 'N06_Psychoanaleptics', 
                      'N05A_Antipsychotics', 'N05B_Anxiolytics', 
                      'N05C_Hypnotics_and_Sedatives', 'N06A_Antidepressants', 
                      'N06B_Psychostimulants_for_ADHD_and_Nootropics', 
                      'N06D_AntiDementia_drugs']

for m in atc_medication_key:
    code = m.split('_')[0]
    users = medications.loc[medications.atc.str.startswith(code)]['id'].unique()
    libr_md.insert(libr_md.shape[1], m, np.where(libr_md.index.isin(users), 'True', 'False'))
    

# exclude anyone who is on antibactials 
#libr_md = libr_md.loc[libr_md['J01_Antibacterials'] == 'False']

In [4]:
libr_md.to_csv('../Updated_Data/Medications_Data/full_libr_md_with_medications.csv')

## Metagenomics metadata cleaning

In [3]:
data_dir = '../Updated_Data/T1000Classic_Qiita10424/'

# metadata cleaning 
md_t1000c = q2.Metadata.load(data_dir + '10424_20230201-070656.txt')
md_t1000c_df = md_t1000c.to_dataframe()

md_t1000c_df['libr_id'] = [np.nan if 'BLANK' in i.upper() else i.split('.')[1].upper() for i in md_t1000c_df.index]
md_t1000c_df['cobre'] = np.where((md_t1000c_df['cobre'] != 'Yes') & (md_t1000c_df['libr_id'].notna()), 
                                 'No', md_t1000c_df['cobre'])
md_t1000c_df['cobre'].replace({'not applicable': np.nan, 'not provided': np.nan},
                              inplace=True)
md_t1000c_df['timepoint'] = np.where(md_t1000c_df.index.str.contains('1year'), 
                                     '1year', 
                                     np.where(md_t1000c_df.libr_id.notna(), 'baseline', 
                                              np.nan))
# remove samples that are not in Tulsa 1000 or COBRE or AS484 -> since this in our metadata 
md_t1000c_filt = md_t1000c_df.loc[(md_t1000c_df['cobre'] == 'Yes') | 
                                  (md_t1000c_df['tulsa_1000'] == 'yes') | 
                                  (md_t1000c_df['libr_id'] == 'AS484')]
# remove non fecal samples
md_t1000c_fecal = md_t1000c_filt.loc[md_t1000c_filt.sample_type == 'feces']

# match to samples from LIBR
md_t1000c_libr = md_t1000c_fecal.loc[md_t1000c_fecal.libr_id.isin(libr_md.index)]

# save for qiime2 
md_t1000c_q2 = q2.Metadata(md_t1000c_libr)

## T1000 Classic Metagenomics Data (qiita ID 10424) 

In [10]:
# MetaG preps to merge 
metag_preps_paths = {
    'prep_12718': data_dir + 'MetaG/156550_ft.qza',
    'prep_12719': data_dir + 'MetaG/156547_ft.qza',
    'prep_12720': data_dir + 'MetaG/156565_ft.qza',
    'prep_12721': data_dir + 'MetaG/156571_ft.qza',
    'prep_12722': data_dir + 'MetaG/156579_ft.qza'
}

metag_preps = [q2.Artifact.load(metag_preps_paths[i]) for i in metag_preps_paths.keys()]
merged_wol2 = feature_table.methods.merge(tables=metag_preps, 
                                          overlap_method='sum').merged_table

# 16S preps to merge 
amp_preps_paths = {
    'prep_1428': data_dir + 'Amplicon/131547_feature-table.qza',
    'prep_2145': data_dir + 'Amplicon/128379_feature-table.qza',
    'prep_2569': data_dir + 'Amplicon/131073_feature-table.qza',
    'prep_4004': data_dir + 'Amplicon/128520_feature-table.qza',
    'prep_4607': data_dir + 'Amplicon/131180_feature-table.qza',
    'prep_4685': data_dir + 'Amplicon/130023_feature-table.qza', 
    'prep_6173': data_dir + 'Amplicon/132180_feature-table.qza', 
    'prep_7243': data_dir + 'Amplicon/132690_feature-table.qza', 
    'prep_8287': data_dir + 'Amplicon/133462_feature-table.qza'
}

preps_16s = [q2.Artifact.load(amp_preps_paths[i]) for i in amp_preps_paths.keys()]
merged_16s = feature_table.methods.merge(tables=preps_16s).merged_table

## Filter to samples only present in LIBR 

In [11]:
table_metag = feature_table.methods.filter_samples(table=merged_wol2, 
                                                   metadata=md_t1000c_q2).filtered_table
table_16s = feature_table.methods.filter_samples(table=merged_16s, 
                                                 metadata=md_t1000c_q2).filtered_table
# also filter 16s to greengenes2 
gg2_tax = q2.Artifact.load('../Updated_Data/2022.10.taxonomy.asv.nwk.qza')
gg2_tree = q2.Artifact.load('../Updated_Data/2022.10.phylogeny.asv.nwk.qza')
table_16s = greengenes2.methods.filter_features(feature_table=table_16s, 
                                                reference=gg2_tree).filtered_feature_table

In [12]:
print('Before removing duplicates: ' + str(table_16s.view(pd.DataFrame).shape))

def merge_with_libr(md_df):
    out = md_df.reset_index().merge(libr_md.reset_index(), left_on='libr_id', 
                                    right_on='id')
    dropped = out.drop(columns=['id']).set_index('index')
    
    for c in dropped.columns[dropped.columns.str.contains('DX_')]: 
        dropped[c] = dropped[c].astype('str')
    
    return dropped.rename_axis(index='SampleID')

# get dataframe representation of tables 
table_metag_df = table_metag.view(pd.DataFrame)
table_16s_df = table_16s.view(pd.DataFrame)

# remove duplicates from 16s 
md_t1000c_libr.insert(loc=md_t1000c_libr.shape[1], column='sequencing_depth', 
                      value=table_16s_df.sum(axis=1))
md_t1000c_libr = md_t1000c_libr.loc[md_t1000c_libr.sequencing_depth.notna()]
md_t1000c_sorted = md_t1000c_libr.sort_values('sequencing_depth', ascending=False)
md_t1000c_dedup = md_t1000c_sorted.loc[~md_t1000c_sorted.libr_id.duplicated()]
md_t1000c_dedup = md_t1000c_dedup.loc[md_t1000c_dedup.sequencing_depth >= 5000]

# get separate metadata for metag and 16s 
metag_md = md_t1000c_libr.reindex(md_t1000c_libr.index.intersection(table_metag_df.index))
metag_md.drop(columns=['sequencing_depth'], inplace=True)
metag_md.insert(loc=metag_md.shape[1], column='sequencing_depth', 
                value=table_metag_df.sum(axis=1))
metag_md = merge_with_libr(metag_md)
metag_md = metag_md.loc[metag_md.sequencing_depth > 500000]
md_metag_q2 = q2.Metadata(metag_md)

md_16s = md_t1000c_dedup.reindex(md_t1000c_dedup.index.intersection(table_16s_df.index))
md_16s = merge_with_libr(md_16s)
md_16s_q2 = q2.Metadata(md_16s)

libr_16s_filt = feature_table.methods.filter_samples(table=table_16s, 
                                                     metadata=md_16s_q2).filtered_table
libr_metag_filt = feature_table.methods.filter_samples(table=table_metag, 
                                                       metadata=md_metag_q2).filtered_table
print('After removing duplicates: ' + str(libr_16s_filt.view(pd.DataFrame).shape))

Before removing duplicates: (962, 17648)
After removing duplicates: (670, 14110)


## Match to LIBR IDs and Nightingale samples

In [6]:
# nightingale 
ng = pd.read_csv('../Updated_Data/Nightingale/Tulsa_Serum_Nightingale_Preprocessed.csv', 
                 index_col=0).T
# reverse log transform 
ng = 2**ng

# diagnosis metadata from LIBR 
md_t1000c_ng = md_t1000c_libr.loc[md_t1000c_libr['libr_id'].isin(ng.index)]

In [7]:
# save the nightingale 
ng.rename_axis(index='SampleID', inplace=True)
ng_cols_to_keep = ng.columns[~(ng.columns.str.contains('pct')) & ~(ng.columns.str.contains('by'))]
ng_bt = q2.Artifact.import_data('FeatureTable[Frequency]', ng).view(biom.Table)
ng_bt.filter(ng_cols_to_keep, axis='observation')
f = h5py.File('../Updated_Data/Microbiome_Nightingale_Integration/nightingale.biom', 'w')
ng_bt.to_hdf5(f, 'tulsa')
f.close()

In [13]:
def get_numbers(md): 
    criteria = {
        'ANX': md.loc[md.DX_ANX == 'True'], 
        'MDD': md.loc[md.DX_MDD == 'True'],
        'MDD&ANX': md.loc[(md.DX_MDD == 'True') & (md.DX_ANX == 'True')],
        'MDD|ANX': md.loc[(md.DX_MDD == 'True') | (md.DX_ANX == 'True')], 
        'MDD&~ANX': md.loc[(md.DX_MDD == 'True') & (md.DX_ANX == 'False')], 
        'ANX&~MDD': md.loc[(md.DX_ANX == 'True') & (md.DX_MDD == 'False')], 
        'HC': md.loc[md.DX_HC == 'True'], 
        'Cobre': md.loc[md.cobre == 'Yes'], 
        'T1000': md.loc[md.cobre == 'No']
    }
    
    out_df = pd.DataFrame(columns=['Number', 'Age', 'Sex (% Female)', 'Medication (% Yes)', 'BMI', 
                                   'C10_LipidReducingAgents', 'N05_Psycholeptics', 'N05A_Antipsychotics', 
                                   'N05B_Anxiolytics', 'N05C_Hypnotics_and_Sedatives', 'N06_Psychoanaleptics', 
                                   'N06A_Antidepressants', 'N06B_Psychostimulants_for_ADHD_and_Nootropics'], 
                          index=criteria.keys())
    
    for k in criteria: 
        out_df['Number'][k] = criteria[k].shape[0]
        out_df['Sex (% Female)'][k] = np.round(criteria[k]['Gender'].value_counts(normalize=True)['Female']*100, 2)
        out_df['Age'][k] = (str(np.round(criteria[k]['Age'].mean(), 2)) + ' ± ' + 
                            str(np.round(criteria[k]['Age'].std(), 2))) 
        out_df['Medication (% Yes)'][k] = np.round(criteria[k]['Medicated'].value_counts(normalize=True)['Medicated']*100, 2)
        out_df['BMI'][k] = (str(np.round(criteria[k]['InBody_BMI'].mean(), 2)) + ' ± ' + 
                            str(np.round(criteria[k]['InBody_BMI'].std(), 2)))
        out_df['C10_LipidReducingAgents'][k] = 100-np.round(criteria[k]['C10_LipidReducingAgents'].value_counts(normalize=True)['False']*100, 2)
        out_df['N05_Psycholeptics'][k] = 100-np.round(criteria[k]['N05_Psycholeptics'].value_counts(normalize=True)['False']*100, 2)
        out_df['N05A_Antipsychotics'][k] = 100-np.round(criteria[k]['N05A_Antipsychotics'].value_counts(normalize=True)['False']*100, 2)
        out_df['N05B_Anxiolytics'][k] = 100-np.round(criteria[k]['N05B_Anxiolytics'].value_counts(normalize=True)['False']*100, 2)
        out_df['N05C_Hypnotics_and_Sedatives'][k] = 100-np.round(criteria[k]['N05C_Hypnotics_and_Sedatives'].value_counts(normalize=True)['False']*100, 2)
        out_df['N06_Psychoanaleptics'][k] = np.round(criteria[k]['N06_Psychoanaleptics'].value_counts(normalize=True)['True']*100, 2)
        out_df['N06A_Antidepressants'][k] = np.round(criteria[k]['N06A_Antidepressants'].value_counts(normalize=True)['True']*100, 2)
        out_df['N06B_Psychostimulants_for_ADHD_and_Nootropics'][k] = 100-np.round(criteria[k]['N06B_Psychostimulants_for_ADHD_and_Nootropics'].value_counts(normalize=True)['False']*100, 2)
        #out_df['N06D_AntiDementia_drugs'] = np.round(criteria[k]['N06D_AntiDementia_drugs'].value_counts(normalize=True)['True']*100, 2)
        #out_df['Cobre'][k] = np.round(criteria[k]['cobre'].value_counts(normalize=True)['Yes']*100, 2)
    return out_df

In [14]:
get_numbers(md_16s)

,Number,Age,Sex (% Female),Medication (% Yes),BMI,C10_LipidReducingAgents,N05_Psycholeptics,N05A_Antipsychotics,N05B_Anxiolytics,N05C_Hypnotics_and_Sedatives,N06_Psychoanaleptics,N06A_Antidepressants,N06B_Psychostimulants_for_ADHD_and_Nootropics
ANX,332,33.41 ± 10.52,77.71,46.99,28.04 ± 5.48,4.52,23.49,2.11,18.98,6.93,41.27,40.36,2.71
MDD,471,34.17 ± 11.02,74.52,46.5,28.39 ± 5.55,4.67,19.32,2.34,14.23,5.94,41.61,40.76,3.4
MDD&ANX,301,33.35 ± 10.63,77.41,46.51,28.18 ± 5.41,3.32,21.26,2.33,16.94,5.98,42.19,41.2,2.99
MDD|ANX,502,34.16 ± 10.92,74.9,46.81,28.28 ± 5.59,5.38,20.92,2.19,15.74,6.57,41.04,40.24,3.19
MDD&~ANX,170,35.64 ± 11.56,69.41,46.47,28.75 ± 5.8,7.06,15.88,2.35,9.41,5.88,40.59,40.0,4.12
ANX&~MDD,31,33.97 ± 9.48,80.65,51.61,26.65 ± 6.04,16.13,45.16,0.0,38.71,16.13,32.26,32.26,0.0
HC,164,30.3 ± 10.72,63.41,7.93,26.71 ± 5.25,4.88,0.0,0.0,0.0,0.0,3.05,1.22,1.83
Cobre,157,29.9 ± 10.13,85.35,17.2,26.69 ± 5.44,2.55,6.37,0.0,3.82,3.82,21.66,20.38,3.18
T1000,513,34.26 ± 11.03,67.84,43.66,28.25 ± 5.52,6.04,18.71,2.14,14.42,5.26,34.89,33.92,2.73


In [11]:
get_numbers(md_16s).to_csv('../Updated_Results/demographic_info_nomeds.csv')

In [36]:
# do rarefaction as well! 
libr_metag_rare = feature_table.methods.rarefy(table=libr_metag_filt, 
                                             sampling_depth=500000).rarefied_table
libr_16s_rare = feature_table.methods.rarefy(table=libr_16s_filt,
                                             sampling_depth=5000).rarefied_table

## get GG2 taxonomy

In [14]:
def filter_table(table, metadata, group): 
    md = metadata.to_dataframe()
    
    criteria = {
        'MDD&ANX': md.loc[((md['DX_MDD'] == 'True') & (md['DX_ANX'] == 'True')) | (md['DX_HC'] == 'True')],
        'MDD|ANX': md.loc[((md['DX_MDD'] == 'True') | (md['DX_ANX'] == 'True') | (md['DX_HC'] == 'True'))], 
        'MDD&~ANX': md.loc[((md['DX_MDD'] == 'True') & (md['DX_ANX'] == 'False')) | (md['DX_HC'] == 'True')],
        'ANX&~MDD': md.loc[((md['DX_MDD'] == 'False') & (md['DX_ANX'] == 'True')) | (md['DX_HC'] == 'True')],  
        'ANX': md.loc[(md['DX_ANX'] == 'True') | (md['DX_HC'] == 'True')], 
        'MDD': md.loc[(md['DX_MDD'] == 'True') | (md['DX_HC'] == 'True')],
        'ANX_MDD_NO_HC': md.loc[((md['DX_MDD'] == 'True') & (md['DX_ANX'] == 'False')) | 
                                ((md['DX_MDD'] == 'False') & (md['DX_ANX'] == 'True'))]
    }
    
    md_filt = q2.Metadata(criteria[group])
    
    return feature_table.methods.filter_samples(table=table, metadata=md_filt).filtered_table

tables_for_analysis = { 
    'Amp_MDD_AND_ANX': filter_table(libr_16s_filt, md_16s_q2, 'MDD&ANX'),
    'Amp_MDD_OR_ANX': filter_table(libr_16s_filt, md_16s_q2, 'MDD|ANX'),
    'Amp_MDD_ONLY': filter_table(libr_16s_filt, md_16s_q2, 'MDD&~ANX'),
    'Amp_ANX_ONLY': filter_table(libr_16s_filt, md_16s_q2, 'ANX&~MDD'), 
    'Amp_ANX_MDD_COMP': filter_table(libr_16s_filt, md_16s_q2, 'ANX_MDD_NO_HC'),
    'Amp_MDD': filter_table(libr_16s_filt, md_16s_q2, 'MDD'),
    'Amp_ANX': filter_table(libr_16s_filt, md_16s_q2, 'ANX'), 
    'MetaG_MDD_AND_ANX': filter_table(libr_metag_filt, md_metag_q2, 'MDD&ANX'),
    'MetaG_MDD_OR_ANX': filter_table(libr_metag_filt, md_metag_q2, 'MDD|ANX'),
    'MetaG_MDD_ONLY': filter_table(libr_metag_filt, md_metag_q2, 'MDD&~ANX'),
    'MetaG_ANX_ONLY': filter_table(libr_metag_filt, md_metag_q2, 'ANX&~MDD'), 
    'MetaG_ANX_MDD_COMP': filter_table(libr_metag_filt, md_metag_q2, 'ANX_MDD_NO_HC'),
    'MetaG_MDD': filter_table(libr_metag_filt, md_metag_q2, 'MDD'),
    'MetaG_ANX': filter_table(libr_metag_filt, md_metag_q2, 'ANX')
}

In [15]:
gg2_class_16s = greengenes2.methods.taxonomy_from_table(reference_taxonomy=gg2_tax, 
                                                        table=tables_for_analysis['Amp_MDD_OR_ANX'])
gg2_taxonomy_16s = gg2_class_16s.classification
gg2_taxonomy_16s.view(pd.DataFrame).to_csv('../Updated_Data/gg2_taxonomy_16s.csv')

gg2_class_metag = greengenes2.methods.taxonomy_from_table(reference_taxonomy=gg2_tax, 
                                                          table=tables_for_analysis['MetaG_MDD_OR_ANX'])
gg2_taxonomy_metag = gg2_class_metag.classification

In [16]:
def get_annotation(x, full_length):
    if len(x) == full_length: 
        return np.nan
    else: 
        split = x.split(';')
        start = -1
        while len(split[start]) == 3:
            start -= 1
    return split[start]

In [17]:
# get ng_bt in same format as feature table 
ng_bt.filter(set(md_16s.libr_id) & set(ng_bt.ids()))
libr_to_sampleid = dict(zip(md_16s.libr_id, md_16s.index))
ng_bt.update_ids(libr_to_sampleid)

168 x 612 <class 'biom.table.Table'> with 102816 nonzero entries (100% dense)

In [18]:
# save biom table for uncollapsed 
for p in tables_for_analysis.keys(): 
    bt = tables_for_analysis[p].view(biom.Table)
    
    f = h5py.File('../Updated_Data/RF_Data/LIBR_Data/' + p + '_uncollapsed.biom', 'w')
    bt.to_hdf5(f, 'tulsa')
    f.close()
    
    bt = bt.merge(ng_bt, sample='intersection', observation='union') 
    f = h5py.File('../Updated_Data/RF_Data/LIBR_Nightingale/' + p + '_uncollapsed.biom', 'w')
    bt.to_hdf5(f, 'tulsa')
    f.close()

# collapse the 16S data at the genus level and MetaG data at the species level 
for t in tables_for_analysis.keys(): 
    if 'Amp' in t: 
        collapsed = taxa.methods.collapse(table=tables_for_analysis[t], taxonomy=gg2_taxonomy_16s,
                                          level=6).collapsed_table
        full_len = 23
    elif 'MetaG' in t: 
        collapsed = taxa.methods.collapse(table=tables_for_analysis[t], taxonomy=gg2_taxonomy_metag,
                                          level=7).collapsed_table
        full_len = 27
    collapsed_df = collapsed.view(pd.DataFrame)
    collapsed_df.columns = [get_annotation(i, full_len) for i in collapsed_df.columns]
    
    if len(collapsed_df.columns[collapsed_df.columns.isna()]) != 0: 
        collapsed_df.drop(columns=[np.nan], inplace=True)
    tables_for_analysis[t] = q2.Artifact.import_data('FeatureTable[Frequency]', collapsed_df)
    
# save biom table for collapsed 
for p in tables_for_analysis.keys(): 
    bt = tables_for_analysis[p].view(biom.Table)
    f = h5py.File('../Updated_Data/RF_Data/LIBR_Data/' + p + '_collapsed.biom', 'w')
    bt.to_hdf5(f, 'tulsa')
    f.close()
    
    bt = bt.merge(ng_bt, sample='intersection', observation='union') 
    f = h5py.File('../Updated_Data/RF_Data/LIBR_Nightingale/' + p + '_collapsed.biom', 'w')
    bt.to_hdf5(f, 'tulsa')
    f.close()

metag_md_comp = metag_md.loc[metag_md.index.difference(md_16s.index)]
full_metadata = pd.concat([md_16s, metag_md_comp])
ng_metadata = md_16s.loc[md_16s.index.intersection(ng_bt.ids())]

full_metadata.to_csv('../Updated_Data/RF_Data/LIBR_Data/metadata.tsv', sep='\t')
ng_metadata.to_csv('../Updated_Data/RF_Data/LIBR_Nightingale/metadata.tsv', sep='\t')

## Unmedicated Participants

In [19]:
# get table as biom table 
ft_to_filt = biom.load_table('../Updated_Data/RF_Data/LIBR_Data/Amp_MDD_OR_ANX_uncollapsed.biom')

# change ids based on key 
feat_key_df = pd.read_csv('../Updated_Data/Medications_Data/feature_key.csv', index_col=0)
feat_key_dict = dict(zip(feat_key_df['sequences'], feat_key_df.index))
ft_to_filt.update_ids(feat_key_dict, axis='observation')

# do a prevalence filter of minimum 10 participants 
prevalence = ft_to_filt.pa(inplace=False).to_dataframe().sum(axis=1)
ft_to_filt.filter(prevalence[prevalence > 10].index, axis='observation')

# filter metadata to MDD, ANX, or HC only 
md_filtered = full_metadata.loc[full_metadata.index.intersection(ft_to_filt.ids())]


# remove participants that have Yes for C10, N05, or N06
md_unmed = md_filtered.loc[(md_filtered.C10_LipidReducingAgents == 'False') & 
                           (md_filtered.N05_Psycholeptics == 'False') & 
                           (md_filtered.N06_Psychoanaleptics == 'False')]
md_unmed.to_csv('../Updated_Data/Medications_Data/unmedicated_metadata.tsv', sep='\t')

md_unmed_birdman = md_unmed.loc[md_unmed['InBody_BMI'].notna()]
md_unmed_birdman.to_csv('../Updated_Data/Medications_Data/unmedicated_metadata_birdman.tsv', sep='\t')

# filter table to unmedicated participants only
unmed_table = ft_to_filt.filter(md_unmed_birdman.index, inplace=False)
f = h5py.File('../Updated_Data/Medications_Data/unmedicated_table.biom', 'w')
unmed_table.to_hdf5(f, 'tulsa')
f.close()

# fitler table to only participants with anxiety 
anx_md = md_filtered.loc[(md_filtered['DX_ANX']=='True') & (md_filtered['InBody_BMI'].notna())]
anx_md.to_csv('../Updated_Data/Medications_Data/anx_metadata.tsv', sep='\t')
anx_table = ft_to_filt.filter(anx_md.index, inplace=False)
f = h5py.File('../Updated_Data/Medications_Data/anx_table.biom', 'w')
anx_table.to_hdf5(f, 'tulsa')
f.close()

# fitler table to only participants with depression 
mdd_md = md_filtered.loc[(md_filtered['DX_MDD']=='True') & (md_filtered['InBody_BMI'].notna())]
mdd_md.to_csv('../Updated_Data/Medications_Data/mdd_metadata.tsv', sep='\t')
mdd_table = ft_to_filt.filter(mdd_md.index, inplace=False)
f = h5py.File('../Updated_Data/Medications_Data/mdd_table.biom', 'w')
mdd_table.to_hdf5(f, 'tulsa')
f.close()

## T1000 - U19 Data (qiita ID 10317 / 13847) 

In [138]:
md_13847 = q2.Metadata.load('../../Tulsa_16S/Data/Metadata/13847_20230206-080057.txt')
md_t1ku19_df = md_13847.to_dataframe()
t1ku19_ids = pd.read_csv('../Data/Metadata/t1000_8.9.21.ids')
agp_to_libr = pd.read_csv('../../Tulsa_16S/Data/Metadata/sample_id_to_libr.tsv', sep='\t',
                          index_col='sample_id')['libr_id'].to_dict()

md_t1ku19_df['suffix'] = [i.split('.')[-1] for i in md_t1ku19_df.index]
md_t1ku19_df['libr_id'] = [agp_to_libr[i] if i in agp_to_libr.keys() else np.nan for i in md_t1ku19_df.suffix]
md_t1ku19_df = md_t1ku19_df.loc[(md_t1ku19_df['libr_id'].isin(libr_md['id'].values)) & 
                                (md_t1ku19_df.sample_type=='feces')]

md_13847 = q2.Metadata(md_t1ku19_df)

In [140]:
t1ku19_metag = q2.Artifact.load(ft_dir + '159164_feature-table.qza')

tables_t1ku19_16s = {
    'Prep_11084': q2.Artifact.load(ft_dir + '136087_feature-table.qza'),
    'Prep_11095': q2.Artifact.load(ft_dir + '135992_feature-table.qza')
}
tables_16s = [tables_t1ku19_16s[i] for i in tables_t1ku19_16s.keys()]
merged_t1ku19_16s = feature_table.methods.merge(tables=tables_16s).merged_table

# filter the tables
t1ku19_metag = feature_table.methods.filter_samples(table=t1ku19_metag, 
                                                    metadata=md_13847).filtered_table
t1ku19_16s = feature_table.methods.filter_samples(table=merged_t1ku19_16s, 
                                                  metadata=md_13847).filtered_table

t1ku19_metag = feature_table.methods.filter_features(table=t1ku19_metag, 
                                                     min_samples=28).filtered_table
t1ku19_16s = feature_table.methods.filter_features(table=t1ku19_16s, 
                                                   min_samples=28).filtered_table

t1ku19_metag = feature_table.methods.filter_samples(table=t1ku19_metag, 
                                                    min_frequency=245000).filtered_table
t1ku19_16s = feature_table.methods.filter_samples(table=t1ku19_16s, 
                                                  min_frequency=5000).filtered_table

In [141]:
samples_16s_t1ku19 = t1ku19_16s.view(pd.DataFrame).index
samples_metag_t1ku19 = t1ku19_metag.view(pd.DataFrame).index

md_16s_t1ku19 = md_t1ku19_df.loc[samples_16s_t1ku19]
md_metag_t1ku19 = md_t1ku19_df.loc[samples_metag_t1ku19]

In [142]:
len(md_16s_t1ku19.libr_id.unique())

113

In [143]:
len(md_metag_t1ku19.libr_id.unique())

118

## Combine all samples

In [123]:
all_16s = pd.concat([filt_md_16s, md_16s_t1ku19])
all_metag = pd.concat([filt_md_metag, md_metag_t1ku19])

In [126]:
len(all_16s['libr_id'].unique())

715

In [127]:
len(all_metag['libr_id'].unique())

346

In [164]:
# bod's metadata 
all_bod = pd.read_csv('../../U19_Tulsa/Data/Metadata/q2_stool_T1000_U19_sample_metadata.tsv', 
                      sep='\t', index_col='sample_name')
all_bod_libr = all_bod.loc[all_bod.libr_id.isin(libr_md['id'].values)]

In [167]:
t1ku19_metag_df = q2.Artifact.load('../../U19_Tulsa/Data/MetaG/clean_feature_table.qza').view(pd.DataFrame)
#t1ku19_metag_df.index = ['.'.join(i.split('.')[1:]) for i in t1ku19_metag_df.index]
bod_q2 = q2.Artifact.import_data('FeatureTable[Frequency]', 
                                 t1ku19_metag_df.loc[all_bod_libr.index.intersection(t1ku19_metag_df.index)]) 
feature_table.visualizers.summarize(table=bod_q2, sample_metadata=q2.Metadata.load('../../U19_Tulsa/Data/Metadata/q2_stool_T1000_U19_sample_metadata.tsv')).visualization

<visualization: Visualization uuid: e8d54267-bef2-42ed-a777-e62cdc658710>

In [166]:
t1ku19_metag_df

,G000005825,G000006605,G000006725,G000006745,G000006785,G000006845,G000006865,G000006925,G000006965,G000007085,...,G900156305,G900156415,G900156525,G900156605,G900156615,G900156675,G900156765,G900156885,G900156915,G900163845
000145946,0.0,0.0,0.0,0.0,1.0,0.0,154.0,91.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000147488,0.0,0.0,0.0,0.0,2.0,0.0,107.0,21.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
000147497,0.0,1.0,0.0,0.0,0.0,0.0,276.0,184.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000147494,0.0,0.0,0.0,0.0,0.0,0.0,35.0,22.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000147743,0.0,0.0,0.0,0.0,1.0,0.0,5.0,6906.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X00174066,0.0,15370.0,0.0,0.0,53.0,0.0,31.0,3514.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X00173305,0.0,7.0,0.0,1.0,0.0,2.0,1119.0,3887.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0
X00174428,0.0,4.0,0.0,0.0,1.0,0.0,2.0,182.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X00174568,1.0,0.0,0.0,0.0,3.0,0.0,639.0,21943.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
all_bod

,libr_id,visit,Age,Gender,W.HRatio,MedHx_BMI,OASIS,SCOFF,PHQ,PROMIS_AlcoUseStd,...,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_change,weight_kg,whole_eggs,whole_grain_frequency,covid_depressed2,mental_illness2,covid_anxious2
sample_name,,,,,,,,,,,,,,,,,,,,,
10317.000154892,AA081,V4,43.7,Male,0.909091,32.094595,0,0,0,3.7,...,Never,Never,Never,Remained stable,117.934157,Regularly (3-5 times/week),Occasionally (1-2 times/week),Not at all,No,Not at all
10317.000156084,AA115,V3,51.8,Female,0.783784,22.494537,0,0,0,4.6,...,Never,Never,Never,Remained stable,58.513485,Occasionally (1-2 times/week),Rarely (less than once/week),Not at all,No,Not at all
10317.000150989,AA343,V3,33.8,Female,0.933333,32.277319,14,1,11,NaN,...,Never,Never,Rarely (a few times/month),Remained stable,90.718582,Occasionally (1-2 times/week),Regularly (3-5 times/week),One or more days,Yes,More than two days
10317.000155870,AA576,V4,50.4,Female,0.822222,29.863770,0,0,0,5.7,...,Never,Never,Rarely (a few times/month),Remained stable,77.564388,Rarely (less than once/week),Regularly (3-5 times/week),Not at all,No,Not at all
10317.X00167095,AA582,V9,51.4,Female,0.826087,32.944822,0,0,0,NaN,...,Rarely (a few times/month),Regularly (3-5 times/week),Never,Remained stable,89.357803,Rarely (less than once/week),Regularly (3-5 times/week),Not at all,No,Not at all
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000150200,BI206,V2,38.2,Female,0.849057,40.383976,13,2,13,2.8,...,Never,Never,Never,Increased more than 10 pounds,108.862298,Regularly (3-5 times/week),Regularly (3-5 times/week),One or more days,Yes,More than two days
10317.000148118,BI365,V2,30.6,Male,0.900000,27.517950,11,0,10,2.6,...,Never,Never,Occasionally (1-2 times/week),Remained stable,74.842830,Occasionally (1-2 times/week),Occasionally (1-2 times/week),One or more days,Yes,More than two days
10317.000150240,BI380,V3,29.9,Female,0.888889,22.998535,9,0,11,2.9,...,Daily,Daily,Occasionally (1-2 times/week),Remained stable,63.503007,Regularly (3-5 times/week),Rarely (less than once/week),One or more days,Yes,One or two days
